# Задания по уроку 5.2

In [2]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              RandomTreesEmbedding, VotingClassifier)
from sklearn.linear_model import LogisticRegression

Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [3]:
X, y = load_breast_cancer(return_X_y=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Задачи с проверкой кода

1) Реализовать функцию нахождения самого частого элемента в массиве (голосование)

In [4]:
def get_most_frequent_value(sequence):
    # TODO your code here

In [5]:
get_most_frequent_value([1, 2, 3, 3, 3, 4, 4])

3

2) Реализовать функцию, выполняющую бутстреп.

In [161]:
def bootstrap(values, new_dataset_size):
    """
    Creates a new dataset from the old one using bootstrap.
    
    :arg values: np.array of shape (n_objects, n_features), input objects
    :arg new_dataset_size: int, number of elements in the resulting array
    
    :return: bootstraped_dataset: np.array of shape (new_dataset_size, n_features)
    """
    # TODO your code here
    Result_arr = np.empty((0,values.shape[1]), int)
    for i in range(new_dataset_size):
        Result_arr = np.append(Result_arr, values[np.random.choice(range(0,values.shape[0]),1)], axis = 0)
    return Result_arr

In [163]:
matr = np.array([[1,2,3],[4,5,6],[7,8,9]])

[7 8 9]


In [173]:
bootstrap(matr, 5)

array([[1, 2, 3],
       [7, 8, 9],
       [4, 5, 6],
       [4, 5, 6],
       [7, 8, 9]])

3) Реализовать фукнцию для подсчета произвольной метрики из Scikit-Learn в режиме OOB. Подсказка: `oob_decision_function_`

In [8]:
# Write-up
def estimate_oob_metric(forest, metric):
    """
    Computes any classification metric in the out-of-bag mode.
        :arg forest: Scikit-Learn ensemble model
        :arg metric: callable with two arguments that returns a float
        
        :return oob_metric_value: float
    """
    # TODO your code here

In [9]:
estimate_oob_metric(forest, f1_score)

0.9738219895287957

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [10]:
def compute_metric(clf, X_train=X_train, y_train=y_train,
                   X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [11]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.972222

1) Напишите индекс самого важного признака по мнению случайного леса. Важности признаков указаны в переменной feature_importances_ случайного леса.

2) Увеличьте количество в случайном лесе до 100. Как изменилось качество? Укажите его.

3) Выставите количество деревьев в лесе равным 100 и включите режим подсчета out-of-bag score. Укажите полученное значение метрики accuracy, округленного до 6 знака после запятой.

4) Выключите режим random subspaces в случайном лесе. Укажите качество.

5) Выключите режим бутстрапа и переобучите случайный лес. Укажите качество.

6) Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

Помимо важностей, которые можно определить встроенным в Scikit-Learn способом, существует еще парочка других, основанных на модификации признаков. Некоторые из них находятся в библиотеке `rfpimp`:

In [17]:
! pip install rfpimp

7) При помощи библиотеки rfpimp определите индекс самого важного признака методом drop-column importance. Как классификатор используйте случайный лес со стандартными настройками.  
Важность необходимо оценить по __отложенной__ выборке.

8) При помощи библиотеки rfpimp определите индекс самого важного признака методом column-permutation importance. Как классификатор используйте случайный лес со стандартными настройками.  
Важность необходимо оценить по __подвыборке__ обучающей выборки (сэмплирование выполняется автоматически внутри библиотеки)

9) Мы познакомились со случайным лесом, который комбинирует бутстрап и random subspaces. Теперь познакомимся с методом, который добавляет еще один уровень случайности — экстремальные деревься (Extra Trees). В этом методе сплиты при построении деревьев строятся случайным образом. Замените RandomForestClassifier на ExtraTreesClassifier с такими же настройками. Укажите полученное качество ниже.

10) Познакомимся с Random Trees Embedding — преобразованием, которое отображает объекты в новое пространство признаков. Оно выполняется при помощи построения Extra Trees. Новое пространство признаков — пространство векторов, которые равны по длине количеству листьев деревьев всего леса. Для каждого объекта в этом векторе проставляется единица, если объект попал в лист при выполнении предсказания решающим деревом. Примените его для преобразования признаков, а затем поверх этих признаков обучите логистическую регрессию. Какое получилось качество? Измерьте его функцией `compute_metric`, выставив необходимые аргументы.

11) Объедините hard голосованием случайный лес, extra trees и логистическую регрессию. Для этого можно воспользоваться классом VotingClassifier. Укажите полученное качество.

12) Объедините soft голосованием случайный лес, extra trees и логистическую регрессию. Укажите полученное качество.

13) Объедините с весами `[0.8, 0.2]` soft голосованием extra trees и логистическую регрессию. Укажите полученное качество.